In [11]:
import os

In [12]:
%pwd

'/Users/rahuln/S-6/Basic_NLP/Text-Summarizer'

In [13]:
os.chdir('/Users/rahuln/S-6/Basic_NLP/Text-Summarizer')

In [14]:
%pwd

'/Users/rahuln/S-6/Basic_NLP/Text-Summarizer'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str  # Changed from evaluation_strategy
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [16]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = "steps",  # Changed from evaluation_strategy
            eval_steps = 500,
            save_steps = 500,
            gradient_accumulation_steps = 8
        )

        return model_trainer_config

In [18]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [19]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cpu"  # Using CPU to avoid memory issues
        print(f"Using device: {device}")

        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

        # Set special tokens
        tokenizer.pad_token = tokenizer.eos_token

        # Loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        print("Dataset loaded successfully")
        print(f"Train dataset size: {len(dataset_samsum_pt['train'])}")
        print(f"Validation dataset size: {len(dataset_samsum_pt['validation'])}")

        # Function to preprocess data
        def preprocess_function(examples):
            # Tokenize inputs
            inputs = examples["dialogue"]
            targets = examples["summary"]
            
            model_inputs = tokenizer(
                inputs,
                max_length=128,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

            # Tokenize targets
            labels = tokenizer(
                targets,
                max_length=128,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

            model_inputs["labels"] = labels["input_ids"]
            return model_inputs

        # Preprocess the datasets
        train_dataset = dataset_samsum_pt["train"].map(
            preprocess_function,
            batched=True,
            remove_columns=dataset_samsum_pt["train"].column_names,
            desc="Running tokenizer on train dataset",
        )

        eval_dataset = dataset_samsum_pt["validation"].map(
            preprocess_function,
            batched=True,
            remove_columns=dataset_samsum_pt["validation"].column_names,
            desc="Running tokenizer on validation dataset",
        )

        # Take smaller subset for testing
        train_dataset = train_dataset.select(range(100))
        eval_dataset = eval_dataset.select(range(20))

        print("Datasets preprocessed and ready for training!")
        
        # Training arguments with correct parameter names
        training_args = TrainingArguments(
            output_dir=str(self.config.root_dir),
            num_train_epochs=1,
            warmup_steps=100,
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            learning_rate=5e-5,
            weight_decay=0.01,
            logging_steps=10,
            eval_strategy="steps",  # Changed from evaluation_strategy
            eval_steps=50,
            save_steps=50,
            gradient_accumulation_steps=4,
            save_total_limit=1,
            load_best_model_at_end=True,
            gradient_checkpointing=True
        )

        # Data collator
        data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model_pegasus,
            padding=True
        )

        # Initialize trainer
        trainer = Trainer(
            model=model_pegasus,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=tokenizer,
            data_collator=data_collator
        )
        
        print("Starting training...")
        try:
            trainer.train()
            print("Training completed successfully!")

            print("Saving model...")
            # Save model
            model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
            # Save tokenizer
            tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
            print("Model and tokenizer saved successfully!")
        except Exception as e:
            print(f"Error during training: {str(e)}")
            raise e

In [20]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-08-17 16:35:35,627:INFO:common: yaml file: config/config.yaml loaded successfully]
[2025-08-17 16:35:35,660:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-17 16:35:35,661:INFO:common: created directory at: artifacts]
[2025-08-17 16:35:35,661:INFO:common: created directory at: artifacts/model_trainer]
[2025-08-17 16:35:35,660:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-17 16:35:35,661:INFO:common: created directory at: artifacts]
[2025-08-17 16:35:35,661:INFO:common: created directory at: artifacts/model_trainer]
Using device: cpu
Using device: cpu
Dataset loaded successfully
Train dataset size: 14732
Validation dataset size: 818
Dataset loaded successfully
Train dataset size: 14732
Validation dataset size: 818


Running tokenizer on validation dataset: 100%|██████████| 818/818 [00:00<00:00, 4832.17 examples/s]

Datasets preprocessed and ready for training!



/var/folders/pr/bpz5w8m93nx13j3gs9tf6s6h0000gn/T/ipykernel_7790/3137231241.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/var/folders/pr/bpz5w8m93nx13j3gs9tf6s6h0000gn/T/ipykernel_7790/3137231241.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss


Training completed successfully!
Saving model...
Model and tokenizer saved successfully!
Model and tokenizer saved successfully!


In [21]:
from inspect import signature
print("Available TrainingArguments parameters:")
print(signature(TrainingArguments))

Available TrainingArguments parameters:
(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict[str, Any], str, NoneType] = <fac